## demo mcmc

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import model as model

%matplotlib inline

## define functions and constants

In [22]:
def newtheta(theta):
    a = 0.05
    fa = lambda x:x + a*x*np.random.randn()
    current = theta
    #newkc = mapkc(current.KC.copy())
    #newsig = mapkc(current.sig.copy())
    proposed = current.map(fa)
    #proposed = current.drop(['KC','sig']).map(fa)
    #proposed['KC']=newkc
    #proposed['sigma']=newsig
    return proposed

In [48]:
def chisq(data,model,variance):
    d = data
    m = model
    v = variance

    #for first epoch probability
    fmax = lambda x:(max(x,0.001)**2)
    N1 = m['COUNTS_U1']
    x1 = (m['FIRST_EPOCH_PROB_U1']-data['FIRST_EPOCH_PROB_U1'])**2
    s1 = m['FIRST_EPOCH_PROB_U1'].map(fmax)
    N2 = m['COUNTS_U2']
    x2 = (m['FIRST_EPOCH_PROB_U2']-data['FIRST_EPOCH_PROB_U2'])**2
    s2 = m['FIRST_EPOCH_PROB_U2'].map(fmax)

    #other chi's follow form: chi = ((d-m)**2)/(v**2)
    chi = pd.DataFrame()
    chi['FIRST_EPOCH_PROB_U1'] = (N1*(x1/s1) + N2*(x2/s2))
    chi['MEAN_U1'] = ((m['MEAN_U1']-data['MEAN_U1'])**2)/(variance['MEAN_U1']**2)
    chi['MEAN_U2'] = ((m['MEAN_U2']-data['MEAN_U2'])**2)/(variance['MEAN_U2']**2)
    chi['SD_U1'] = ((m['SD_U1']-data['SD_U1'])**2)/(variance['SD_U1']**2)
    chi['SD_U2'] = ((m['SD_U2']-data['SD_U2'])**2)/(variance['SD_U2']**2)

    return chi

In [24]:
def chisum(df):
    #return scalar
    one = df.sum(axis=0)
    two = one.sum(0)
    return two

In [25]:
def taupen(tau_a):
    penalty = (tau_a-2)**2
    return penalty

def kpen(ks,kc):
    delta = 0
    pen = delta*(ks + kc)**2
    return pen

In [26]:
index = [0,20,40,60,80]
cv = 0.6
cv_var = 0.2
p1_var = 0.1
fcv = lambda x: x*cv
fcv_var = lambda x:(x*cv_var)
fp1_var = lambda x:(x*p1_var)

## load data

In [27]:
syprob = pd.read_csv('sy_prob.csv',index_col=0)
syU1 = pd.read_csv('syU1.csv',index_col=0)
syU2 = pd.read_csv('syU2.csv',index_col=0)

## construct dataframe of data

In [28]:
data = pd.DataFrame()
data['COUNTS_U1'] = syprob['counts_U1']
data['COUNTS_U2'] = syprob['counts_U2']
data['FIRST_EPOCH_PROB_U1'] = syprob['U1']
data['FIRST_EPOCH_PROB_U2'] = syprob['U2']
data['MEAN_U1'] = syU1['MEAN']
data['MEAN_U2'] = syU2['MEAN']
data['SD_U1'] = data['MEAN_U1'].map(fcv)
data['SD_U2'] = data['MEAN_U2'].map(fcv)
data.index=index
col = data.columns

In [29]:
data

,COUNTS_U1,COUNTS_U2,FIRST_EPOCH_PROB_U1,FIRST_EPOCH_PROB_U2,MEAN_U1,MEAN_U2,SD_U1,SD_U2
0,50,50,0.50,0.50,1800.0,1700.0,1080.0,1020.0
20,95,5,0.95,0.05,1900.0,1700.0,1140.0,1020.0
40,99,1,0.99,0.01,2200.0,1700.0,1320.0,1020.0
60,100,0,1.00,0.00,2700.0,1700.0,1620.0,1020.0
80,100,0,1.00,0.00,3200.0,1700.0,1920.0,1020.0


## calculate error and make dataframe from data

In [30]:
variance = pd.DataFrame()
variance['COUNTS_U1'] = data['COUNTS_U1'].map(fp1_var)
variance['COUNTS_U2'] = data['COUNTS_U2'].map(fp1_var)
variance['FIRST_EPOCH_PROB_U1'] = data['FIRST_EPOCH_PROB_U1'].map(fp1_var)
variance['FIRST_EPOCH_PROB_U2'] = data['FIRST_EPOCH_PROB_U2'].map(fp1_var)
variance['MEAN_U1'] = data['MEAN_U1'].map(fcv_var)
variance['MEAN_U2'] = data['MEAN_U2'].map(fcv_var)
variance['SD_U1'] = data['SD_U1'].map(fcv_var)
variance['SD_U2'] = data['SD_U2'].map(fcv_var)
variance.index = index
variance.columns = col

In [31]:
variance

,COUNTS_U1,COUNTS_U2,FIRST_EPOCH_PROB_U1,FIRST_EPOCH_PROB_U2,MEAN_U1,MEAN_U2,SD_U1,SD_U2
0,5.0,5.0,0.050,0.050,360.0,340.0,216.0,204.0
20,9.5,0.5,0.095,0.005,380.0,340.0,228.0,204.0
40,9.9,0.1,0.099,0.001,440.0,340.0,264.0,204.0
60,10.0,0.0,0.100,0.000,540.0,340.0,324.0,204.0
80,10.0,0.0,0.100,0.000,640.0,340.0,384.0,204.0


### load initial parameters

In [32]:
initial = pd.read_pickle('IP2.pkl')
initial

alpha       0.000100
beta        1.127209
gamma       1.777396
kb          0.100947
kc          0.002921
ks          0.228017
mu          1.122287
sigma       0.045653
tau_a    1500.853819
tau_u       8.940977
Name: 173, dtype: float64

In [33]:
!ls

CHANGESmcmc.ipynb      mcmc.py                rivalry.pyc
IP.pkl                 mcmc.pyc               syU1.csv
IP1.pkl                mcmc_pltsavebest.ipynb syU2.csv
IP2.pkl                model.py               sy_bias.csv
README.md              model.pyc              sy_prob.csv
Rivalry.jl             modelequations2.0.py   temp
demodynamics.ipynb     paramset_model.ipynb
initialparameters.txt  rivalry.py


In [34]:
theta_cur = initial
theta_prop = newtheta(theta_cur)

In [35]:
print theta_cur
print theta_prop

alpha       0.000100
beta        1.127209
gamma       1.777396
kb          0.100947
kc          0.002921
ks          0.228017
mu          1.122287
sigma       0.045653
tau_a    1500.853819
tau_u       8.940977
Name: 173, dtype: float64
alpha       0.000100
beta        1.216326
gamma       1.771524
kb          0.100682
kc          0.003002
ks          0.225976
mu          1.182233
sigma       0.042519
tau_a    1568.443399
tau_u       8.796285
Name: 173, dtype: float64


## run model with current and proposed theta values

In [39]:
#trials
dominance = 1
firstprob = 200

In [ ]:
#m1 = model.run(theta_cur,dominance,firstprob)

In [46]:
#m2 = model.run(theta_prop,dominance,firstprob)

In [42]:
m1.index = index
m1.columns = col
m1

,COUNTS_U1,COUNTS_U2,FIRST_EPOCH_PROB_U1,FIRST_EPOCH_PROB_U2,MEAN_U1,MEAN_U2,SD_U1,SD_U2
0,99.0,101.0,0.495,0.505,736.438806,742.800000,443.244235,447.884392
20,138.0,62.0,0.690,0.310,774.246246,719.515060,473.119181,435.856735
40,151.0,49.0,0.755,0.245,818.886154,702.266871,488.692059,417.728461
60,164.0,36.0,0.820,0.180,879.380952,692.615873,506.370315,396.750101
80,188.0,12.0,0.940,0.060,892.391304,650.593168,539.192658,388.652912


In [45]:
m2.index = index
m2.columns=col
m2

,COUNTS_U1,COUNTS_U2,FIRST_EPOCH_PROB_U1,FIRST_EPOCH_PROB_U2,MEAN_U1,MEAN_U2,SD_U1,SD_U2
0,96.0,104.0,0.480,0.520,1521.141975,1512.870370,520.446317,519.955069
20,121.0,79.0,0.605,0.395,1593.728395,1459.968944,541.525087,494.729105
40,127.0,73.0,0.635,0.365,1714.386076,1393.525316,561.673536,445.849618
60,140.0,60.0,0.700,0.300,1833.316129,1343.597403,580.043666,421.527995
80,151.0,49.0,0.755,0.245,1993.738255,1327.255034,572.321722,368.801999


## caluclate chi construct dataframe

In [49]:
chicur = pd.DataFrame(chisq(data,m1,variance),index=index)
chiprop = pd.DataFrame(chisq(data,m2,variance),index=index)

In [50]:
chicur

,FIRST_EPOCH_PROB_U1,MEAN_U1,MEAN_U2,SD_U1,SD_U2
0,0.020002,8.728105,7.925881,8.690370,7.865154
20,63.207106,8.776465,8.316183,8.555133,8.199331
40,59.710772,9.852662,8.611344,9.915532,8.716143
60,43.902439,11.367125,8.778744,11.813853,9.333921
80,12.765957,13.000630,9.526425,12.930155,9.578026


In [53]:
chiprop

,FIRST_EPOCH_PROB_U1,MEAN_U1,MEAN_U2,SD_U1,SD_U2
0,0.320513,0.600014,0.302920,6.710827,6.008385
20,99.612930,0.649600,0.498399,6.890047,6.629890
40,108.747708,1.218083,0.812515,8.250962,7.921200
60,85.714286,2.575929,1.098813,10.302442,8.606515
80,64.900662,3.552411,1.201893,12.317144,10.189803


##  calulate likelihood ratio

In [54]:
sumchicur = chisum(chicur)
sumprop = chisum(chiprop)
print sumchicur
print sumprop

370.087457924
455.633890042


In [55]:
ratio = np.exp((-sumprop+sumchicur)/2)
ratio

2.6535562151025129e-19